In [2]:
import scipy.io
mat = scipy.io.loadmat('imdb_crop/imdb.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'imdb_crop/imdb.mat'

In [4]:
dob = mat['imdb'][0, 0][0]
photo_taken = mat['imdb'][0, 0][1]
full_path = mat['imdb'][0, 0][2]
gender = mat['imdb'][0, 0][3]

from datetime import datetime, timedelta
import numpy as np


train_len = 430000
test_len = dob.shape[1] - train_len

age = np.zeros_like(dob)
false_age = 0

for i in range(train_len):
    matlab_datenum = float(dob[0][i])
    try:
      python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
      age[0][i] = np.abs(python_datetime.year - photo_taken[0][i])
    except:
      false_age += 1
      age[0][i] = 20

print(false_age)

111


In [0]:
import os
import shutil

for i in range(0, 20):
    directory = 'E:\\Projects\\Age_prediction\\data\\train\\'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    directory = 'E:\\Projects\\Age_prediction\\data\\test\\'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [0]:
import random
n = dob.shape[1]

index = random.sample(range(0, n), n)

for i in range(train_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'E:\\Projects\\Age_prediction\\data\\train\\'+str(int(age[0][index[i]]/5))     
    src = "E:\\Projects\\Age_prediction\\imdb_crop\\"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)


for i in range(train_len, train_len + test_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'E:\\Projects\\Age_prediction\\data\\test\\'+str(int(age[0][index[i]]/5))         
    src = "E:\\Projects\\Age_prediction\\imdb_crop\\"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)
    

# Building the model

In [0]:
import os, shutil
train_dir = 'data/train'
test_dir = 'data/test'

In [8]:
!pip install -q keras
import keras

Using TensorFlow backend.


In [9]:
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

Found 401249 images belonging to 20 classes.
Found 28670 images belonging to 20 classes.


In [10]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))


58892288/58889256 [==============================] - 4s 0us/step


In [11]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10260     
Total params: 18,919,764
Trainable params: 18,919,764
Non-trainable params: 0
_________________________________________________________________


In [0]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=12600, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=900)